In [3]:
#import libraries
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

#Create Variables( spotify credentials and playlist link )
cid = '00b7317977ad4c0d971af8274f1aa790'
secret = '6efbf45fe72d435f9739d0c0f4c26db5'
playlist_link = 'https://open.spotify.com/playlist/37i9dQZF1DX0XUsuxWHRQd'

#authorization
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#DErivationof the playlistID
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
print(playlist_URI)
result = [x["track"]["artists"][0]for x in sp.playlist_tracks(playlist_URI)["items"]]
# results = [x["artists"]for x in sp.artists("artist_id")]

print(result)


37i9dQZF1DX0XUsuxWHRQd
[{'external_urls': {'spotify': 'https://open.spotify.com/artist/46SHBwWsqBkxI7EeeBEQG7'}, 'href': 'https://api.spotify.com/v1/artists/46SHBwWsqBkxI7EeeBEQG7', 'id': '46SHBwWsqBkxI7EeeBEQG7', 'name': 'Kodak Black', 'type': 'artist', 'uri': 'spotify:artist:46SHBwWsqBkxI7EeeBEQG7'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/31W5EY0aAly4Qieq6OFu6I'}, 'href': 'https://api.spotify.com/v1/artists/31W5EY0aAly4Qieq6OFu6I', 'id': '31W5EY0aAly4Qieq6OFu6I', 'name': 'A Boogie Wit da Hoodie', 'type': 'artist', 'uri': 'spotify:artist:31W5EY0aAly4Qieq6OFu6I'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/45TgXXqMDdF8BkjA83OM7z'}, 'href': 'https://api.spotify.com/v1/artists/45TgXXqMDdF8BkjA83OM7z', 'id': '45TgXXqMDdF8BkjA83OM7z', 'name': 'Rod Wave', 'type': 'artist', 'uri': 'spotify:artist:45TgXXqMDdF8BkjA83OM7z'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/757aE44tKEUQEqRuT6GnEB'}, 'href': 'https://api.spotify.com/v1/art

In [4]:
table_ft_list=["track_id","track_name","artist_id","artist_name","album_id","album_name","album_release_date","album_type","track_popularity","track_explicit"]
playlist_df = pd.DataFrame(columns = table_ft_list)
playlist = sp.playlist_tracks(playlist_URI)["items"]
for track in playlist:
  playlistft_dict ={}

  playlistft_dict["track_id"]= track["track"]["id"]
  playlistft_dict["track_name"]= track["track"]["name"]
  playlistft_dict["artist_id"]= track["track"]["artists"][0]["id"]
  playlistft_dict["artist_name"]= track["track"]["artists"][0]["name"]
  playlistft_dict["album_id"]= track["track"]["album"]["id"]
  playlistft_dict["album_name"]= track["track"]["album"]["name"]
  playlistft_dict["album_release_date"]= track["track"]["album"]["release_date"]
  playlistft_dict["album_type"]= track["track"]["album"]["type"]
  playlistft_dict["track_popularity"]= track["track"]["popularity"]
  playlistft_dict["track_explicit"]= track["track"]["explicit"]

  # Concat the dfs
  track_df = pd.DataFrame(playlistft_dict, index = [0])
  playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
  


In [5]:
artiste_ft_list=["artist_id","artist_name","popularity","followers"]
artist_df = pd.DataFrame(columns = artiste_ft_list)
Artist_id =  [x["track"]["artists"][0]["uri"]for x in sp.playlist_tracks(playlist_URI)["items"]]
Artist_id = list(set(Artist_id))
for uri  in Artist_id:
  artistft_dict ={}

  artistft_dict["artist_id"]= sp.artist(uri)["id"]
  artistft_dict["artist_name"]=sp.artist(uri)["name"]
  artistft_dict["popularity"]=sp.artist(uri)["popularity"]
  artistft_dict["followers"]= sp.artist(uri)["followers"]["total"]



  # Concat the dfs
  artistft_df = pd.DataFrame(artistft_dict, index = [0])
  artist_df = pd.concat([artist_df, artistft_df], ignore_index = True)

In [25]:
artist_df.head(1)
# playlist_df.head(1)

,artist_id,artist_name,popularity,followers
0,6AgTAQt8XS6jRWi4sX7w49,Polo G,92,6230995


In [8]:
import sqlite3
conn = sqlite3.connect('Rap Caviar') 
c = conn.cursor()
# c.execute('''
#           drop TABLE artists
#           ''')  
# c.execute('''
#           drop TABLE tracks
#           ''')         
playlist_df.to_sql('tracks', conn)
artist_df.to_sql('artists', conn)

# tracks = pd.read_sql('SELECT * FROM tracks', conn)   

conn.commit()

# **1).Most_albums_are_released_on**

In [10]:
most_albums_are_released_on  = pd.read_sql('SELECT strftime("%w", album_release_date)Day, count(*) Num_releases FROM tracks group by strftime("%w", album_release_date) order by count(*) desc  LIMIT 1 ', conn)
print (most_albums_are_released_on)

  Day  Num_releases
0   5            41


# **2).Which artists have 3+ tracks? Sort your results by the number of tracks from highest to lowest**



In [11]:
Artist_with_3_or_more_tracks  = pd.read_sql('SELECT artist_name, count(*) Num_releases FROM tracks group by artist_name having count(*)>= 3 order by count(*) desc', conn)
print (Artist_with_3_or_more_tracks)

   artist_name  Num_releases
0  Kodak Black             3
1       Polo G             3
2  Roddy Ricch             3


#**3. Which tracks have both a track and artist popularity of at least 90?**#


In [34]:
Tracks_with_track_and_popularity_90  = pd.read_sql('select a.track_name, a.artist_name,a.track_popularity from tracks a where track_popularity>= 90 and exists(select 1 from artists b  where b.popularity >=90 and a.artist_id = b.artist_id)'  , conn)
print (Tracks_with_track_and_popularity_90)

                                     track_name   artist_name  track_popularity
0                    Don’t Play (with Lil Baby)        Polo G                79
1                                   ESCAPE PLAN  Travis Scott                85
2                         Rockstar In His Prime    Juice WRLD                82
3                                         MAFIA  Travis Scott                82
4            Bubbly (with Drake & Travis Scott)    Young Thug                82
5   Knife Talk (with 21 Savage ft. Project Pat)         Drake                89
6                Fair Trade (with Travis Scott)         Drake                88
7                                    Praise God    Kanye West                88
8                     Bad Man (Smooth Criminal)        Polo G                75
9                          Feelin' Like Tunechi     Lil Wayne                72
10                                 Young N Dumb        Polo G                77
11                                 Alrea

#**4). What are the top 3 months for album releases?**

In [64]:
from pandas.core import groupby
#Add athe month of releas column to te dataframe
playlist_df['album_release_month'] = pd.DatetimeIndex(playlist_df['album_release_date']).month

#dataframe of tracks by album_release_month
playlist_df_by_month =playlist_df['album_release_month'].value_counts()


#Top 3 album_release_month by count
playlist_df_by_month.head(3)



12    17
10    14
11    10
Name: album_release_month, dtype: int64

#**5. Are clean or explicit songs more popular on average?**

In [44]:
playlist_df.head(2)
playlist_df.groupby(['track_explicit']).sum(['track_popularity'])


#From the output of the above, Explicit songs have a higher popularity

,album_release_month
track_explicit,
False,8
True,522


#**6. Is there a relationship between track and artist popularity? If so, what is it?**


In [88]:
#merge the arist and playlist datataframs
result = pd.merge(playlist_df, artist_df, how="left", on=["artist_id", "artist_id"])
result.head(100)

# result.corr()

result['popularity'] = result['popularity'].astype(float, errors = 'raise')
result['track_popularity'] = result['track_popularity'].astype(float, errors = 'raise')


correlation = result['track_popularity'].corr(result['popularity'])
print(correlation)



0.42972053131824134
